In [ ]:
!nvcc --version
%pip install nvcc4jupyter
%load_ext nvcc4jupyter

In [4]:
%%cuda
#include <stdio.h>
#include <stdlib.h>

#define N 32
#define THREADS_PER_BLOCK 8

__global__ 
void parallelSearch(int *arr, int target, int *result) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (idx < N) {
        if (arr[idx] == target) {
            result[0] = idx;
        }
    }
}

int main() {
    int h_array[N];
    int target = 42;
    int h_result[1] = {-1};

    for (int i = 0; i < N; i++) {
        h_array[i] = rand() % 50;
    }

    printf("Array:\n");
    for (int i = 0; i < N; i++) {
        printf("%d ", h_array[i]);
    }
    printf("\n");

    int *d_array, *d_result;
    cudaMalloc(&d_array, N * sizeof(int));
    cudaMalloc(&d_result, sizeof(int));

    cudaMemcpy(d_array, h_array, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_result, h_result, sizeof(int), cudaMemcpyHostToDevice);

    int blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    parallelSearch<<<blocks, THREADS_PER_BLOCK>>>(d_array, target, d_result);
    cudaDeviceSynchronize();

    cudaMemcpy(h_result, d_result, sizeof(int), cudaMemcpyDeviceToHost);

    if (h_result[0] != -1)
        printf("Target %d found at index %d\n", target, h_result[0]);
    else
        printf("Target %d not found.\n", target);

    cudaFree(d_array);
    cudaFree(d_result);

    return 0;
}


Array:
41 17 34 0 19 24 28 8 12 14 5 45 31 27 11 41 45 42 27 36 41 4 2 3 42 32 21 16 18 45 47 26 
Target 42 found at index 24

